# The Oxford-IIIT Pet Dataset
- https://www.robots.ox.ac.uk/~vgg/data/pets/
- 37개 카테고리의 개, 고양이 품종 데이터셋. 각 클래스 별로 대략 200여장의 이미지를 제공한다.
- 파일명에 품종이 포함되어 있어 classification 데이터셋을 만들 수 있다.
- 3686개 이미지에대한 annotation 파일을 제공한다.
    - bounding box는 각 pet의 얼굴을 가리킨다. 

In [ ]:
# library import
import os
 
import re
import random
import xml.etree.ElementTree as et
from PIL import Image
 
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras
 
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
 
print(tf.__version__)
print(keras.__version__)

# Oxford Pet Dataset
- 파일명
    - 품종명_번호.jpg
    - 대문자로 시작: 고양이, 소문자로 시작: 개
    - Egyptian_Mau_1.jpg, american_bulldog_10.jpg

### 데이터셋 다운로드

In [1]:
!pip install gdown

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Created wheel for gdown: filename=gdown-3.13.0-py3-none-any.whl size=9034 sha256=e989f3bea30e4b83864dd65b8b4a8d9f054a94a7d606df35a97196e333ab9f28
  Stored in directory: c:\users\zions\appdata\local\pip\cache\wheels\2f\2a\2f\86449b6bdbaa9aef873f68332b68be6bfbc386b9219f47157d
Successfully built gdown


In [3]:
import gdown
url = 'https://drive.google.com/uc?id=1gXqmWrxJqdp_luNKZmv81vY5cjFLuTdT'
fname = 'oxford_pet.zip'
gdown.download(url, fname, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1gXqmWrxJqdp_luNKZmv81vY5cjFLuTdT
To: C:\Users\zions\12.ObjectDetection\oxford_pet.zip

0.00B [00:00, ?B/s]
524kB [00:00, 1.91MB/s]
1.57MB [00:00, 4.18MB/s]
2.62MB [00:00, 4.63MB/s]
3.15MB [00:00, 4.27MB/s]
4.19MB [00:00, 5.22MB/s]
5.77MB [00:01, 7.20MB/s]
7.34MB [00:01, 8.57MB/s]
8.91MB [00:01, 9.16MB/s]
10.5MB [00:01, 9.58MB/s]
12.2MB [00:01, 10.5MB/s]
13.8MB [00:01, 10.9MB/s]
15.3MB [00:01, 10.0MB/s]
16.9MB [00:02, 11.2MB/s]
18.5MB [00:02, 11.6MB/s]
20.1MB [00:02, 11.1MB/s]
21.9MB [00:02, 11.9MB/s]
23.5MB [00:02, 10.9MB/s]
25.1MB [00:02, 11.9MB/s]
26.6MB [00:02, 10.9MB/s]
28.2MB [00:03, 11.4MB/s]
29.8MB [00:03, 10.8MB/s]
31.4MB [00:03, 11.4MB/s]
33.0MB [00:03, 10.4MB/s]
35.1MB [00:03, 11.8MB/s]
36.6MB [00:03, 10.9MB/s]
38.7MB [00:03, 11.9MB/s]
40.2MB [00:04, 11.1MB/s]
41.8MB [00:04, 11.7MB/s]
43.4MB [00:04, 10.6MB/s]
45.0MB [00:04, 11.3MB/s]
46.5MB [00:04, 10.6MB/s]
47.9MB [00:04, 10.6MB/s]
49.5MB [00:04, 11.7MB/s]
51.1MB [00:05, 1

470MB [00:42, 11.1MB/s]
472MB [00:42, 9.29MB/s]
473MB [00:42, 9.17MB/s]
474MB [00:43, 10.5MB/s]
476MB [00:43, 10.9MB/s]
477MB [00:43, 10.2MB/s]
478MB [00:43, 11.1MB/s]
480MB [00:43, 11.5MB/s]
482MB [00:43, 11.6MB/s]
483MB [00:43, 10.1MB/s]
485MB [00:43, 12.1MB/s]
487MB [00:44, 12.0MB/s]
488MB [00:44, 11.2MB/s]
490MB [00:44, 12.1MB/s]
492MB [00:44, 11.0MB/s]
493MB [00:44, 11.8MB/s]
495MB [00:44, 12.0MB/s]
496MB [00:44, 11.3MB/s]
498MB [00:45, 11.5MB/s]
499MB [00:45, 10.7MB/s]
501MB [00:45, 11.5MB/s]
503MB [00:45, 10.5MB/s]
504MB [00:45, 11.1MB/s]
506MB [00:45, 10.3MB/s]
507MB [00:45, 11.1MB/s]
509MB [00:46, 11.8MB/s]
510MB [00:46, 11.4MB/s]
512MB [00:46, 10.3MB/s]
514MB [00:46, 11.8MB/s]
516MB [00:46, 11.0MB/s]
518MB [00:46, 11.8MB/s]
519MB [00:47, 12.1MB/s]
521MB [00:47, 11.2MB/s]
522MB [00:47, 11.5MB/s]
524MB [00:47, 10.4MB/s]
526MB [00:47, 12.2MB/s]
528MB [00:47, 10.9MB/s]
529MB [00:47, 11.2MB/s]
531MB [00:48, 11.7MB/s]
532MB [00:48, 10.5MB/s]
534MB [00:48, 10.6MB/s]
535MB [00:48, 10

'oxford_pet.zip'

In [ ]:
!rm -r oxford_pet

In [ ]:
## 압축풀기
!unzip -q oxford_pet.zip -d oxford_pet

In [ ]:
## 압축이 풀린 directory 확인
!ls oxford_pet

In [ ]:
# 경로 설정
base_dir = '/content/oxford_pet' #데이터셋 기본경로
image_dir = os.path.join(base_dir, 'images') # image 디렉토리 경로
bbox_dir = os.path.join(base_dir, 'annotations', 'xmls') #annotation 파일경로

In [ ]:
# 이미지 파일명 조회
image_files = [fname for fname in os.listdir(image_dir) if os.path.splitext(fname)[-1] == '.jpg']
print(len(image_files))
 
image_files[:3]

In [ ]:
#annotation 파일 경로
bbox_files = [fname for fname in os.listdir(bbox_dir) if os.path.splitext(fname)[-1] == '.xml']
print(len(bbox_files))
bbox_files[:3]

In [ ]:
# 이미지 파일중 RGB가 아닌 이미지 파일과 그 파일에 대한 annotation파일 제거
remove_image_cnt = 0
for image_file in image_files:
    image_path = os.path.join(image_dir, image_file)
    bbox_file = os.path.splitext(image_file)[0]+'.xml'
    bbox_path = os.path.join(bbox_dir, bbox_file)
    
    image = Image.open(image_path)
    image_mode = image.mode
    if image_mode != 'RGB':
        image = np.asarray(image)
        print(image_file, image_mode, image.shape)

        os.remove(image_path)
        remove_image_cnt += 1
        try:
            os.remove(bbox_path)
            print(bbox_path)
        except FileNotFoundError:
            pass

In [ ]:
# 삭제후 image, annotation 파일 목록 다시만들기
image_files = [fname for fname in os.listdir(image_dir) if os.path.splitext(fname)[-1] == '.jpg']
bbox_files = [fname for fname in os.listdir(bbox_dir) if os.path.splitext(fname)[-1] == '.xml']

### class dictionary 생성

In [ ]:
# 클래스 딕셔너리 리스트로 만들기
class_list = set() 
for image_file in image_files:
    file_name = os.path.splitext(image_file)[0] 
    class_name = re.sub('_\d+', '', file_name)
    class_list.add(class_name)    
class_list = list(class_list) 
class_list.sort()  
print(len(class_list))

In [ ]:
# 리스트 딕셔너리로 만들기: class->index로 반환하는 것.
class2idx = {cls:idx for idx, cls in enumerate(class_list)}
class2idx

# train/validation 데이터셋 만들기

## TFRecord 만들기

In [ ]:
IMG_SIZE = 224
N_BBOX = len(bbox_files)
N_TRAIN = 3000 
N_VAL = N_BBOX - N_TRAIN 

In [ ]:
## TFRecord 저장할 directory 생성
tfr_dir = os.path.join(base_dir, 'tfrecord')
os.makedirs(tfr_dir, exist_ok=True)

tfr_train_dir = os.path.join(tfr_dir, 'oxford_train.tfr')
tfr_val_dir = os.path.join(tfr_dir, 'oxford_val.tfr')

In [ ]:
## TFRecord writer 생성
writer_train = tf.io.TFRecordWriter(tfr_train_dir)
writer_val = tf.io.TFRecordWriter(tfr_val_dir)

In [ ]:
# The following functions can be used to convert a value to a type compatible with tf.Example.
 
def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))
 
def _float_feature(value):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))
 
def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [ ]:
# train, validation 데이터셋의 index 설정

# 데이터 섞기
shuffle_list = list(range(N_BBOX))
random.shuffle(shuffle_list) 
# 분할
train_idx_list = shuffle_list[:N_TRAIN]
val_idx_list = shuffle_list[N_TRAIN:]

In [ ]:
# Train TFRecord 생성
for idx in train_idx_list:
    bbox_file = bbox_files[idx]
    bbox_path = os.path.join(bbox_dir, bbox_file)
 
    tree = et.parse(bbox_path)
    width = float(tree.find('./size/width').text)
    height = float(tree.find('./size/height').text)
    xmin = float(tree.find('./object/bndbox/xmin').text)
    ymin = float(tree.find('./object/bndbox/ymin').text)
    xmax = float(tree.find('./object/bndbox/xmax').text)
    ymax = float(tree.find('./object/bndbox/ymax').text)
    #X, Y Center 좌표
    xc = (xmin + xmax) / 2.
    yc = (ymin + ymax) / 2.
 
    x = xc / width
    y = yc / height

    w = (xmax - xmin) / width
    h = (ymax - ymin) / height
 
    file_name = os.path.splitext(bbox_file)[0]
    image_file = file_name + '.jpg'
    image_path = os.path.join(image_dir, image_file)
    image = Image.open(image_path)
    image = image.resize((IMG_SIZE, IMG_SIZE))
    bimage = image.tobytes()
 
    class_name = re.sub('_\d+', '', file_name)
    class_num = class2idx[class_name]
    
    
    if file_name[0].islower():
        bi_cls_num = 0
    else:
        bi_cls_num = 1
    
    
    example = tf.train.Example(features=tf.train.Features(feature={
          'image': _bytes_feature(bimage),
          'cls_num': _int64_feature(class_num),
          'bi_cls_num': _int64_feature(bi_cls_num),
          'x': _float_feature(x),
          'y': _float_feature(y),
          'w': _float_feature(w),
          'h': _float_feature(h)
    }))
    writer_train.write(example.SerializeToString())

writer_train.close()

In [ ]:
# Validation TFRecord 생성
for idx in val_idx_list:
    bbox_file = bbox_files[idx]
    bbox_path = os.path.join(bbox_dir, bbox_file)
 
    tree = et.parse(bbox_path)
    width = float(tree.find('./size/width').text)
    height = float(tree.find('.size/height').text)
    xmin = float(tree.find('./object/bndbox/xmin').text)
    ymin = float(tree.find('./object/bndbox/ymin').text)
    xmax = float(tree.find('./object/bndbox/xmax').text)
    ymax = float(tree.find('./object/bndbox/ymax').text)
    xc = (xmin + xmax) / 2.
    yc = (ymin + ymax) / 2.
    x = xc / width
    y = yc / height
    w = (xmax - xmin) / width
    h = (ymax - ymin) / height
 
    file_name = os.path.splitext(bbox_file)[0]
    image_file = file_name + '.jpg'
    image_path = os.path.join(image_dir, image_file)
    image = Image.open(image_path)
    image = image.resize((IMG_SIZE, IMG_SIZE))
    bimage = image.tobytes()
 
    class_name = re.sub('_\d+', '', file_name)
    class_num = class2idx[class_name]
 
    if file_name[0].islower():
        bi_cls_num = 0
    else:
        bi_cls_num = 1
    
    
    example = tf.train.Example(features=tf.train.Features(feature={
      'image': _bytes_feature(bimage),
      'cls_num': _int64_feature(class_num),
      'bi_cls_num': _int64_feature(bi_cls_num),
      'x': _float_feature(x),
      'y': _float_feature(y),
      'w': _float_feature(w),
      'h': _float_feature(h)
    }))
    writer_val.write(example.SerializeToString())

writer_val.close()

# Localization 모델 학습

In [ ]:
## Hyper Parameters
LEARNING_RATE = 0.0001
N_CLASS = len(class_list)
N_EPOCHS = 40
N_BATCH = 40
IMG_SIZE = 224

steps_per_epoch = N_TRAIN // N_BATCH
validation_steps = int(np.ceil(N_VAL / N_BATCH)) 

print(steps_per_epoch, validation_steps)

In [ ]:
# TFRecord에 저장된 Dataset의 하나의 Data를 parsing하는 함수
def _parse_function(tfrecord_serialized):
    """
    [매개변수]
        tfrecord_serialized: parsing할 1개의 data
    [반환값] 
        튜플 (image, ground truth)
    """
    
    features={'image': tf.io.FixedLenFeature([], tf.string),
              'cls_num': tf.io.FixedLenFeature([], tf.int64),
              'bi_cls_num': tf.io.FixedLenFeature([], tf.int64),
              'x': tf.io.FixedLenFeature([], tf.float32),
              'y': tf.io.FixedLenFeature([], tf.float32),
              'w': tf.io.FixedLenFeature([], tf.float32),
              'h': tf.io.FixedLenFeature([], tf.float32)              
             }
    
    parsed_features = tf.io.parse_single_example(tfrecord_serialized, features)
    
    
    image = tf.io.decode_raw(parsed_features['image'], tf.uint8)    
    image = tf.reshape(image, [IMG_SIZE, IMG_SIZE, 3])
    image = tf.cast(image, tf.float32)/255.
    
    cls_label = tf.cast(parsed_features['cls_num'], tf.int64)
    bi_cls_label = tf.cast(parsed_features['bi_cls_num'], tf.int64)
    
    x = tf.cast(parsed_features['x'], tf.float32)
    y = tf.cast(parsed_features['y'], tf.float32)
    w = tf.cast(parsed_features['w'], tf.float32)
    h = tf.cast(parsed_features['h'], tf.float32)
    grount_truth = tf.stack([x, y, w, h], -1)
    
    return image, grount_truth

In [ ]:
## train dataset 만들기
train_dataset = tf.data.TFRecordDataset(tfr_train_dir)
train_dataset = train_dataset.map(_parse_function, num_parallel_calls=tf.data.experimental.AUTOTUNE)
train_dataset = train_dataset.shuffle(buffer_size=N_TRAIN).prefetch(tf.data.experimental.AUTOTUNE).batch(N_BATCH).repeat()

In [ ]:
## validation dataset 만들기
val_dataset = tf.data.TFRecordDataset(tfr_val_dir)
val_dataset = val_dataset.map(_parse_function, num_parallel_calls=tf.data.experimental.AUTOTUNE)
val_dataset = val_dataset.batch(N_BATCH)

### Trainset의 데이터 읽어서 bounding box 확인

In [ ]:
for image, gt in val_dataset.take(3):
    
    '''그림을 그리기 위해서 bbox의 왼쪽 위 꼭지점 좌표를 계산하고, 
    xmin, ymin, w, h 각각을 image size에 맞게 scaling'''
    x = gt[:,0] 
    y = gt[:,1] 
    w = gt[:,2] 
    h = gt[:,3] 
    xmin = x[0].numpy() - w[0].numpy()/2.
    ymin = y[0].numpy() - h[0].numpy()/2.
    rect_x = int(xmin * IMG_SIZE) 
    rect_y = int(ymin * IMG_SIZE)
    rect_w = int(w[0].numpy() * IMG_SIZE)
    rect_h = int(h[0].numpy() * IMG_SIZE)
    
    
    rect = Rectangle((rect_x, rect_y), rect_w, rect_h, fill=False, color='red')
    plt.axes().add_patch(rect)
    plt.imshow(image[0])
    plt.show()

## 모델 생성 및 학습


In [ ]:
from tensorflow.keras import optimizers
from tensorflow.keras.applications import ResNet101V2
from tensorflow.keras.layers import Conv2D, ReLU, MaxPooling2D, Dense, BatchNormalization, GlobalAveragePooling2D, Concatenate
from tensorflow import keras

In [ ]:
def create_l_model():
    resnet101v2 = ResNet101V2(include_top=False, weights='imagenet', input_shape=(IMG_SIZE, IMG_SIZE, 3))
    model = keras.models.Sequential()
    model.add(resnet101v2)
    model.add(GlobalAveragePooling2D())
    model.add(Dense(256))
    model.add(BatchNormalization())
    model.add(ReLU())
    model.add(Dense(64))
    model.add(BatchNormalization())
    model.add(ReLU())
    model.add(Dense(4, activation='sigmoid'))
    return model

In [ ]:
model = create_l_model()
model.summary()

In [ ]:
# 모델 컴파일
## learning rate scheduing
lr_schedule = keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=LEARNING_RATE,
                                                          decay_steps=steps_per_epoch*10,
                                                          decay_rate=0.5,
                                                          staircase=True)
model.compile(optimizers.Adam(lr_schedule), loss="mse")

In [ ]:
filepath = r'/content/drive/MyDrive/save_models/oxford_pet_localization_resnet101v2_model'
mc_callback = keras.callbacks.ModelCheckpoint(filepath, 'val_loss',verbose=1, save_best_only=True)
es_callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=1)

history = model.fit(train_dataset, steps_per_epoch=steps_per_epoch,
                    epochs=N_EPOCHS,
                    validation_data=val_dataset,
                    validation_steps=validation_steps, 
                    callbacks=[mc_callback, es_callback])

## 확인

In [ ]:
# 미리학습한 모델 다운로드
import gdown
url = 'https://drive.google.com/uc?id=1-2IbiHp3SdffxkqIj4iGL9-recS6g697'
fname = 'oxford_pet_localization_resnet101.tar.gz'
gdown.download(url, fname, quiet=False)

In [ ]:
!mkdir models

In [ ]:
!tar -zxvf oxford_pet_localization_resnet101.tar.gz -C  models

In [ ]:
# 저장된 모델 load
filepath = '/content/models/oxford_pet_localization_resnet101v2_model'
saved_model = keras.models.load_model(filepath)

In [ ]:
saved_model.summary()

### Bounding Box 그리기

In [ ]:
# 예측한 bounding box와 ground truth box를 image에 같이 표시
# 정답은 빨간색 box, 예측은 파란색 box
idx = 0
num_imgs = validation_steps
for val_data, val_gt in val_dataset.take(num_imgs):

    x = val_gt[:,0]
    y = val_gt[:,1]
    w = val_gt[:,2]
    h = val_gt[:,3]

    xmin = x[idx].numpy() - w[idx].numpy()/2.
    ymin = y[idx].numpy() - h[idx].numpy()/2.

    rect_x = int(xmin * IMG_SIZE)
    rect_y = int(ymin * IMG_SIZE)
    rect_w = int(w[idx].numpy() * IMG_SIZE)
    rect_h = int(h[idx].numpy() * IMG_SIZE)
    

    rect = Rectangle((rect_x, rect_y), rect_w, rect_h, fill=False, color='red')
    plt.axes().add_patch(rect)
    
    prediction = saved_model.predict(val_data)
    pred_x = prediction[:,0]
    pred_y = prediction[:,1]
    pred_w = prediction[:,2]
    pred_h = prediction[:,3]
    pred_xmin = pred_x[idx] - pred_w[idx]/2.
    pred_ymin = pred_y[idx] - pred_h[idx]/2.
    pred_rect_x = int(pred_xmin * IMG_SIZE)
    pred_rect_y = int(pred_ymin * IMG_SIZE)
    pred_rect_w = int(pred_w[idx] * IMG_SIZE)
    pred_rect_h = int(pred_h[idx] * IMG_SIZE)
    
    pred_rect = Rectangle((pred_rect_x, pred_rect_y), pred_rect_w, pred_rect_h,
                         fill=False, color='blue')
    plt.axes().add_patch(pred_rect)
    
    
    plt.imshow(val_data[idx])
    plt.show()
 


### IoU확인하기

In [ ]:
## Valiation set의 IOU 계산
avg_iou = 0
num_imgs = validation_steps
res = N_VAL % N_BATCH
for i, (val_data, val_gt) in enumerate(val_dataset.take(num_imgs)):
    
    flag = (i == validation_steps-1)
    x = val_gt[:,0]
    y = val_gt[:,1]
    w = val_gt[:,2]
    h = val_gt[:,3]
    prediction = saved_model.predict(val_data)
    pred_x = prediction[:,0]
    pred_y = prediction[:,1]
    pred_w = prediction[:,2]
    pred_h = prediction[:,3]
    for idx in range(N_BATCH):
        if(flag):
            if idx == res:
                flag = False
                break  
        
        xmin = int((x[idx].numpy() - w[idx].numpy()/2.)*IMG_SIZE)
        ymin = int((y[idx].numpy() - h[idx].numpy()/2.)*IMG_SIZE)
        xmax = int((x[idx].numpy() + w[idx].numpy()/2.)*IMG_SIZE)
        ymax = int((y[idx].numpy() + h[idx].numpy()/2.)*IMG_SIZE)
        
        pred_xmin = int((pred_x[idx] - pred_w[idx]/2.)*IMG_SIZE)
        pred_ymin = int((pred_y[idx] - pred_h[idx]/2.)*IMG_SIZE)
        pred_xmax = int((pred_x[idx] + pred_w[idx]/2.)*IMG_SIZE)
        pred_ymax = int((pred_y[idx] + pred_h[idx]/2.)*IMG_SIZE)
 
        if xmin > pred_xmax or xmax < pred_xmin:        
            continue
        if ymin > pred_ymax or ymax < pred_ymin:        
            continue
        
        gt_width = xmax-xmin
        gt_height = ymax - ymin
        pred_width = pred_xmax - pred_xmin
        pred_height = pred_ymax - pred_ymin
 
        inter_width  =  np.min((xmax, pred_xmax)) - np.max((xmin, pred_xmin))
        inter_height = np.min((ymax, pred_ymax)) - np.max((ymin, pred_ymin))
  
 
        iou = (inter_width * inter_height)/((gt_width * gt_height) + (pred_width * pred_height) - (inter_width * inter_height))

        avg_iou += iou / N_VAL

print(avg_iou)

# Classification을 추가하여 Multi-task Learning으로 Localization 학습하기

- **고양이/개 2개 class로 classification**

In [ ]:
from tensorflow.keras import optimizers
from tensorflow.keras.applications import ResNet101V2
from tensorflow.keras.layers import Conv2D, ReLU, MaxPooling2D, Dense, BatchNormalization, GlobalAveragePooling2D, Concatenate
from tensorflow import keras

In [ ]:
#tfrecord 파싱 함수(classification + localization)
def _parse_function(tfrecord_serialized):
    features={'image': tf.io.FixedLenFeature([], tf.string),
              'cls_num': tf.io.FixedLenFeature([], tf.int64),
              'bi_cls_num': tf.io.FixedLenFeature([], tf.int64),
              'x': tf.io.FixedLenFeature([], tf.float32),
              'y': tf.io.FixedLenFeature([], tf.float32),
              'w': tf.io.FixedLenFeature([], tf.float32),
              'h': tf.io.FixedLenFeature([], tf.float32)              
             }
    parsed_features = tf.io.parse_single_example(tfrecord_serialized, features)
    
    image = tf.io.decode_raw(parsed_features['image'], tf.uint8)    
    image = tf.reshape(image, [IMG_SIZE, IMG_SIZE, 3])
    image = tf.cast(image, tf.float32)/255.

    cls_label = tf.cast(parsed_features['cls_num'], tf.float32)
    bi_cls_label = tf.cast(parsed_features['bi_cls_num'], tf.float32)
    
    x = tf.cast(parsed_features['x'], tf.float32)
    y = tf.cast(parsed_features['y'], tf.float32)
    w = tf.cast(parsed_features['w'], tf.float32)
    h = tf.cast(parsed_features['h'], tf.float32)
    ground_truth = tf.stack([bi_cls_label, x, y, w, h], -1) 
    
    return image, ground_truth

In [ ]:
#Train Dataset 생성
train_dataset = tf.data.TFRecordDataset(tfr_train_dir)
train_dataset = train_dataset.map(_parse_function, num_parallel_calls=tf.data.experimental.AUTOTUNE)
train_dataset = train_dataset.shuffle(buffer_size=N_TRAIN).prefetch(
    tf.data.experimental.AUTOTUNE).batch(N_BATCH).repeat()

In [ ]:
#Validation Dataset 생성
val_dataset = tf.data.TFRecordDataset(tfr_val_dir)
val_dataset = val_dataset.map(_parse_function, num_parallel_calls=tf.data.experimental.AUTOTUNE)
val_dataset = val_dataset.batch(N_BATCH).repeat()

In [ ]:
def create_cl_model():    
    resnet101v2 = ResNet101V2(include_top=False, weights='imagenet', input_shape=(IMG_SIZE, IMG_SIZE, 3))
    gap = GlobalAveragePooling2D()(resnet101v2.output)

    dense_b1_1 = Dense(256)(gap)
    bn_b1_2 = BatchNormalization()(dense_b1_1)
    relu_b1_3 = ReLU()(bn_b1_2)
    dense_b1_4 = Dense(64)(relu_b1_3)
    bn_b1_5 = BatchNormalization()(dense_b1_4)
    relu_b1_6 = ReLU()(bn_b1_5)
    output1 = Dense(2, activation='softmax', name='output1')(relu_b1_6)

    dense_b2_1 = Dense(256)(gap)
    bn_b2_2 = BatchNormalization()(dense_b2_1)
    relu_b2_3 = ReLU()(bn_b2_2)
    dense_b2_4 = Dense(64)(relu_b2_3)
    bn_b2_5 = BatchNormalization()(dense_b2_4)
    relu_b2_6 = ReLU()(bn_b2_5)
    output2 = Dense(4, activation='sigmoid', name='output2')(relu_b2_6)
    
    concat = Concatenate(name='finaly_output')([output1, output2])
    return keras.Model(inputs=resnet101v2.input, outputs=concat)

In [ ]:
model = create_cl_model()
model.summary()

In [ ]:
keras.utils.plot_model(model, show_shapes=True)

In [ ]:
# Loss 함수 구현
def loss_fn(y_true, y_pred):
    cls_labels = tf.cast(y_true[:,:1], tf.int64)
    loc_labels = y_true[:,1:]
    cls_preds = y_pred[:,:2]
    loc_preds = y_pred[:,2:]
    cls_loss = tf.keras.losses.SparseCategoricalCrossentropy()(cls_labels, cls_preds)
    loc_loss = tf.keras.losses.MeanSquaredError()(loc_labels, loc_preds)
    return cls_loss + 5*loc_loss

In [ ]:
# 모델 컴파일
## learning rate scheduing
lr_schedule = keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=LEARNING_RATE,
                                                          decay_steps=steps_per_epoch*10,
                                                          decay_rate=0.5,
                                                          staircase=True)
model.compile(optimizers.Adam(lr_schedule), loss=loss_fn)

In [ ]:
# 학습
filepath2 = r'/content/drive/MyDrive/save_models/oxford_pet_localization_classification_resnet101v2_weights/oxford_pet_lc_weights.ckpt'
mc_callback = keras.callbacks.ModelCheckpoint(filepath2, 'val_loss',verbose=1, save_best_only=True, save_weights_only=True)
es_callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=1)

history = model.fit(train_dataset, steps_per_epoch=steps_per_epoch,
                    epochs=N_EPOCHS,
                    validation_data=val_dataset,
                    validation_steps=validation_steps,
                    callbacks=[mc_callback, es_callback])

# 확인

### 미리 학습된 weights 가져오기 평가하기

In [ ]:
# 미리학습한 모델 다운로드
import gdown
url = 'https://drive.google.com/uc?id=1ycRNri9Gr6QjcOFv4GQi_DLCU17JbOyo'
fname = 'oxford_pet_classification_localization_resnet101_weight.tar.gz'
gdown.download(url, fname, quiet=False)

In [ ]:
# 압축풀기
!tar -zxvf oxford_pet_classification_localization_resnet101_weight.tar.gz -C  models

In [ ]:
# 마지막으로 저장된 checkpoint 경로 확인
best_weight_path = tf.train.latest_checkpoint('/content/models/oxford_pet_localization_classification_resnet101v2_weights')
best_weight_path

In [ ]:
# 저장된 weight load 
saved_model2 = create_cl_model()
saved_model2.load_weights(best_weight_path)

### Bounding Box 확인

In [ ]:
# 배치중 idx번째 것만 확인
idx = 1
num_imgs = validation_steps 
for val_data, val_gt in val_dataset.take(num_imgs):
    
   
    gt_cls_name = np.where(val_gt[:,0]==0,'dog','cat')
    
    x = val_gt[:,1]
    y = val_gt[:,2]
    w = val_gt[:,3]
    h = val_gt[:,4]
    xmin = x[idx].numpy() - w[idx].numpy()/2.
    ymin = y[idx].numpy() - h[idx].numpy()/2.
    rect_x = int(xmin * IMG_SIZE)
    rect_y = int(ymin * IMG_SIZE)
    rect_w = int(w[idx].numpy() * IMG_SIZE)
    rect_h = int(h[idx].numpy() * IMG_SIZE)
    
    rect = Rectangle((rect_x, rect_y), rect_w, rect_h, fill=False, color='red')
    plt.axes().add_patch(rect)    
    
    prediction = saved_model2.predict(val_data)
    
    pred_cls_idx = np.argmax(prediction[:,:2], axis=-1)
    pred_cls_name = np.where(pred_cls_idx==0, 'dog','cat')
    
    pred_x = prediction[:,2]
    pred_y = prediction[:,3]
    pred_w = prediction[:,4]
    pred_h = prediction[:,5]
    pred_xmin = pred_x[idx] - pred_w[idx]/2.
    pred_ymin = pred_y[idx] - pred_h[idx]/2.
    pred_rect_x = int(pred_xmin * IMG_SIZE)
    pred_rect_y = int(pred_ymin * IMG_SIZE)
    pred_rect_w = int(pred_w[idx] * IMG_SIZE)
    pred_rect_h = int(pred_h[idx] * IMG_SIZE)
    
    pred_rect = Rectangle((pred_rect_x, pred_rect_y), pred_rect_w, pred_rect_h,
                         fill=False, color='blue')
    plt.axes().add_patch(pred_rect)
    plt.title(f'Ground Truth-{gt_cls_name[idx]}, Pred:{pred_cls_name[idx]}')
    
    plt.imshow(val_data[idx])
    plt.show()

### IoU 계산

In [ ]:
avg_iou = 0
num_imgs = validation_steps
res = N_VAL % N_BATCH
for i, (val_data, val_gt) in enumerate(val_dataset.take(num_imgs)):    
    flag = (i == validation_steps-1)
    x = val_gt[:,1]
    y = val_gt[:,2]
    w = val_gt[:,3]
    h = val_gt[:,4]

    prediction = saved_model2.predict(val_data)
    
    pred_x = prediction[:,2]
    pred_y = prediction[:,3]
    pred_w = prediction[:,4]
    pred_h = prediction[:,5]
    for idx in range(N_BATCH):
        if(flag):
            if idx == res:
                flag = False
                break          
        xmin = int((x[idx].numpy() - w[idx].numpy()/2.)*IMG_SIZE)
        ymin = int((y[idx].numpy() - h[idx].numpy()/2.)*IMG_SIZE)
        xmax = int((x[idx].numpy() + w[idx].numpy()/2.)*IMG_SIZE)
        ymax = int((y[idx].numpy() + h[idx].numpy()/2.)*IMG_SIZE)
        
        pred_xmin = int((pred_x[idx] - pred_w[idx]/2.)*IMG_SIZE)
        pred_ymin = int((pred_y[idx] - pred_h[idx]/2.)*IMG_SIZE)
        pred_xmax = int((pred_x[idx] + pred_w[idx]/2.)*IMG_SIZE)
        pred_ymax = int((pred_y[idx] + pred_h[idx]/2.)*IMG_SIZE)
 
        if xmin > pred_xmax or xmax < pred_xmin:        
            continue
        if ymin > pred_ymax or ymax < pred_ymin:        
            continue
        
        gt_width = xmax-xmin
        gt_height = ymax - ymin
        pred_width = pred_xmax - pred_xmin
        pred_height = pred_ymax - pred_ymin
 
        inter_width  =  np.min((xmax, pred_xmax)) - np.max((xmin, pred_xmin))
        inter_height = np.min((ymax, pred_ymax)) - np.max((ymin, pred_ymin))
  
 
        iou = (inter_width * inter_height)/((gt_width * gt_height) + (pred_width * pred_height) - (inter_width * inter_height))
        avg_iou += iou / N_VAL
        
print(avg_iou)

# 새로운 Image로 Test하기

In [ ]:
from PIL import Image
image = Image.open('dog.jpg')
image = image.resize((224, 224))
image = np.array(image)
image = image/255.
image = image[np.newaxis, ...]
print(image.shape)

In [ ]:
# 예측 결과 확인 - bounding box, class
from matplotlib.patches import Rectangle
prediction = saved_model2.predict(image)
pred_cls = np.where(np.argmax(prediction[0,:2], axis=-1)==0, 'dog', 'cat')
print(pred_cls)
pred_x = prediction[0,2]
pred_y = prediction[0,3]
pred_w = prediction[0,4]
pred_h = prediction[0,5]
pred_xmin = pred_x - pred_w/2.
pred_ymin = pred_y - pred_h/2.
pred_rect_x = int(pred_xmin * IMG_SIZE)
pred_rect_y = int(pred_ymin * IMG_SIZE)
pred_rect_w = int(pred_w * IMG_SIZE)
pred_rect_h = int(pred_h * IMG_SIZE)

pred_rect = Rectangle((pred_rect_x, pred_rect_y), pred_rect_w, pred_rect_h,
                       fill=False, color='red')
plt.axes().add_patch(pred_rect)

import matplotlib.pyplot as plt
plt.imshow(image[0])
plt.title(f'Prediction class:{pred_cls}')
plt.show()

# TFR 파일 생성 practice 

### 데이터셋 다운로드

In [19]:
# !pip install opencv-contrib-python

In [18]:
import os
import re
import random
import xml.etree.ElementTree as et
from PIL import Image
import cv2
import tensorflow as tf
import numpy as np

In [4]:
!pip install gdown

In [5]:
import gdown
url = 'https://drive.google.com/uc?id=1gXqmWrxJqdp_luNKZmv81vY5cjFLuTdT'
fname = 'oxford_pet.zip'
gdown.download(url, fname, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1gXqmWrxJqdp_luNKZmv81vY5cjFLuTdT
To: C:\Users\zions\12.ObjectDetection\oxford_pet.zip

0.00B [00:00, ?B/s]
524kB [00:00, 2.31MB/s]
1.57MB [00:00, 4.53MB/s]
2.10MB [00:00, 4.47MB/s]
3.67MB [00:00, 7.65MB/s]
5.24MB [00:00, 7.91MB/s]
7.34MB [00:00, 10.2MB/s]
9.07MB [00:01, 10.0MB/s]
10.6MB [00:01, 11.1MB/s]
11.9MB [00:01, 11.5MB/s]
13.2MB [00:01, 10.2MB/s]
14.7MB [00:01, 10.2MB/s]
16.3MB [00:01, 10.7MB/s]
17.8MB [00:01, 11.3MB/s]
19.4MB [00:02, 11.5MB/s]
21.0MB [00:02, 11.1MB/s]
22.5MB [00:02, 12.0MB/s]
24.1MB [00:02, 11.4MB/s]
25.7MB [00:02, 12.1MB/s]
27.3MB [00:02, 11.5MB/s]
29.0MB [00:02, 12.4MB/s]
30.6MB [00:02, 11.8MB/s]
32.1MB [00:03, 11.9MB/s]
33.7MB [00:03, 10.8MB/s]
35.3MB [00:03, 10.3MB/s]
37.4MB [00:03, 11.1MB/s]
38.7MB [00:03, 9.02MB/s]
40.3MB [00:03, 9.39MB/s]
41.9MB [00:04, 10.3MB/s]
43.4MB [00:04, 8.01MB/s]
45.4MB [00:04, 10.0MB/s]
46.9MB [00:04, 10.6MB/s]
48.5MB [00:04, 10.4MB/s]
50.0MB [00:04, 10.6MB/s]
51.6MB [00:05, 1

463MB [00:42, 12.2MB/s]
464MB [00:42, 10.3MB/s]
466MB [00:42, 11.3MB/s]
467MB [00:42, 11.0MB/s]
469MB [00:42, 12.3MB/s]
471MB [00:42, 12.3MB/s]
473MB [00:42, 12.2MB/s]
474MB [00:43, 11.3MB/s]
476MB [00:43, 12.1MB/s]
477MB [00:43, 11.3MB/s]
479MB [00:43, 12.1MB/s]
480MB [00:43, 9.26MB/s]
482MB [00:43, 9.88MB/s]
483MB [00:44, 9.82MB/s]
485MB [00:44, 11.2MB/s]
487MB [00:44, 10.9MB/s]
488MB [00:44, 11.8MB/s]
490MB [00:44, 11.3MB/s]
491MB [00:44, 11.9MB/s]
493MB [00:44, 11.6MB/s]
494MB [00:44, 11.6MB/s]
496MB [00:45, 11.5MB/s]
498MB [00:45, 11.9MB/s]
499MB [00:45, 11.8MB/s]
501MB [00:45, 11.4MB/s]
502MB [00:45, 9.81MB/s]
504MB [00:45, 9.79MB/s]
505MB [00:46, 10.2MB/s]
507MB [00:46, 10.4MB/s]
509MB [00:46, 10.4MB/s]
510MB [00:46, 11.1MB/s]
512MB [00:46, 10.8MB/s]
513MB [00:46, 11.8MB/s]
515MB [00:46, 11.0MB/s]
516MB [00:46, 11.9MB/s]
518MB [00:47, 11.2MB/s]
520MB [00:47, 11.9MB/s]
521MB [00:47, 11.1MB/s]
523MB [00:47, 12.0MB/s]
524MB [00:47, 11.6MB/s]
526MB [00:47, 11.5MB/s]
527MB [00:47, 11

'oxford_pet.zip'

### 데이터셋 전처리

In [2]:
base_dir = './oxford_pet' # 데이터셋 기본경로
image_dir = os.path.join(base_dir, 'images')
bbox_dir = os.path.join(base_dir, 'annotations','xmls')
print(base_dir, image_dir, bbox_dir)

./oxford_pet ./oxford_pet\images ./oxford_pet\annotations\xmls


In [4]:
os.listdir(image_dir)

['Abyssinian_1.jpg',
 'Abyssinian_10.jpg',
 'Abyssinian_100.jpg',
 'Abyssinian_101.jpg',
 'Abyssinian_102.jpg',
 'Abyssinian_103.jpg',
 'Abyssinian_104.jpg',
 'Abyssinian_105.jpg',
 'Abyssinian_106.jpg',
 'Abyssinian_107.jpg',
 'Abyssinian_108.jpg',
 'Abyssinian_109.jpg',
 'Abyssinian_11.jpg',
 'Abyssinian_110.jpg',
 'Abyssinian_111.jpg',
 'Abyssinian_112.jpg',
 'Abyssinian_113.jpg',
 'Abyssinian_114.jpg',
 'Abyssinian_115.jpg',
 'Abyssinian_116.jpg',
 'Abyssinian_117.jpg',
 'Abyssinian_118.jpg',
 'Abyssinian_119.jpg',
 'Abyssinian_12.jpg',
 'Abyssinian_120.jpg',
 'Abyssinian_121.jpg',
 'Abyssinian_122.jpg',
 'Abyssinian_123.jpg',
 'Abyssinian_124.jpg',
 'Abyssinian_125.jpg',
 'Abyssinian_126.jpg',
 'Abyssinian_127.jpg',
 'Abyssinian_128.jpg',
 'Abyssinian_129.jpg',
 'Abyssinian_13.jpg',
 'Abyssinian_130.jpg',
 'Abyssinian_131.jpg',
 'Abyssinian_132.jpg',
 'Abyssinian_133.jpg',
 'Abyssinian_134.jpg',
 'Abyssinian_135.jpg',
 'Abyssinian_136.jpg',
 'Abyssinian_137.jpg',
 'Abyssinian_138.

In [5]:
# 이미지 파일명 마지막을 기준으로 확장자가 jpg, xml 인 파일들만 조회
image_files = [fname for fname in os.listdir(image_dir)
               if os.path.splitext(fname)[-1] == '.jpg']
bbox_file = [fname for fname in os.listdir(bbox_dir)
            if os.path.splitext(fname)[-1] == '.xml']

In [7]:
# 이미지 파일 중 RGB가 아닌 파일과 그 파일의 annotation파일 제거
remove_image_cnt = 0
for image_file in image_files:
    image_path = os.path.join(image_dir, image_file)
    bbox_file = os.path.splitext(image_file)[0]+'.xml'
    bbox_path = os.path.join(bbox_dir, bbox_file)
    
    image = Image.open(image_path) # 이미지파일 읽기
    image_mode = image.mode # 이미지 색공간 string으로 저장
    if image_mode != 'RGB':
        image = np.asarray(image)
        os.remove(image_path)
        remove_image_cnt += 1
        try:
            os.remove(bbox_path)
        except FileNotFoundError:
            pass

In [6]:
image_path

NameError: name 'image_path' is not defined

In [8]:
remove_image_cnt

12

In [9]:
# 삭제 후 image, annotation 파일 목록 다시 작성
image_files = [fname for fname in os.listdir(image_dir)
               if os.path.splitext(fname)[-1]=='.jpg']
bbox_files = [fname for fname in os.listdir(bbox_dir)
               if os.path.splitext(fname)[-1]=='.xml']

In [10]:
len(image_files), len(bbox_files)

(7378, 3685)

### class dictionary 생성

In [12]:
# 클래스 딕셔너리를 리스트로 변환
class_list = set()
for image_file in image_files:
    file_name = os.path.splitext(image_file)[0]
    class_name = re.sub('_\d+', '', file_name)
    class_list.add(class_name)
class_list = list(class_list)
class_list.sort()

In [13]:
# 리스트 딕셔너리로 만들기
class2idx = {cls:idx for idx,cls in enumerate(class_list)}

### train/validation 데이터셋 만들기

In [14]:
IMG_SIZE = 224
N_BBOX = len(bbox_files)
N_TRAIN = 3000
N_VAL = N_BBOX - N_TRAIN

In [15]:
# TFRecord 저장할 directory 생성
tfr_dir = os.path.join(base_dir, 'tfrecord')
os.makedirs(tfr_dir, exist_ok=True)

tfr_train_dir = os.path.join(tfr_dir, 'oxford_train.tfr')
tfr_val_dir = os.path.join(tfr_dir, 'oxford_val.tfr')

In [33]:
# TFRecord writer 생성
writer_train = tf.io.TFRecordWriter(tfr_train_dir)
wrtier_val = tf.io.TFRecordWriter(tfr_val_dir)

In [34]:
# 값을 Feature로 변환하는 함수
def _bytes_feature(value):
    if isinstance(value, type(tf.constant(0))): # 만약 텐서플로우의 텐서라면 넘파이배열로 변환
        value = value.numpy()
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [35]:
# train, validation 데이터셋의 index 설정

# 데이터 섞기
shuffle_list = list(range(N_BBOX))
random.shuffle(shuffle_list)
# 분할
train_idx_list = shuffle_list[:N_TRAIN]
val_idx_list = shuffle_list[N_TRAIN:]

In [38]:
# Train TFRecord 생성
for idx in train_idx_list:
    bbox_file = bbox_files[idx]
    bbox_path = os.path.join(bbox_dir, bbox_file)
    # annotation(root태그) - size - width,height
    # annotation - bndbox - xmin,ymin,xmax,ymax
    tree = et.parse(bbox_path) # 현재 위치: root태그
    width = float(tree.find('./size/width').text)
    height = float(tree.find('./size/height').text)
    xmin = float(tree.find('./object/bndbox/xmin').text)
    ymin = float(tree.find('./object/bndbox/ymin').text)
    xmax = float(tree.find('./object/bndbox/xmax').text)
    ymax = float(tree.find('./object/bndbox/ymax').text)
    xc = (xmin+xmax)/2.
    yc = (ymin+ymax)/2.
    x = xc/width
    y = yc/width
    w = (xmax-xmin)/width
    h = (ymax-ymin)/height
    
    # 이미지 파일
    file_name = os.path.splitext(bbox_file)[0]
    image_file = file_name + '.jpg'
    imge_path = os.path.join(image_dir, image_file)
    image = Image.open(image_path)
    image = image.resize((IMG_SIZE,IMG_SIZE))
    bimage = image.tobytes()
    
    class_name = re.sub('_\d+', '', file_name)
    class_num = class2idx[class_name]
    
    if file_name[0].islower():
        bi_cls_num = 0
    else:
        bi_cls_num = 1
        
        example = tf.train.Example(features=tf.train.Features(feature={
            'image': _bytes_feature(bimage),
            'cls_num': _int64_feature(class_num),
            'bi_cls_num': _int64_feature(bi_cls_num),
            'x': _float_feature(x),
            'y': _float_feature(y),
            'w': _float_feature(w),
            'h': _float_feature(h),
        }))
        writer_train.write(example.SerializeToString())
        
    writer_train.close()

FailedPreconditionError: Writer not initialized or previously closed

In [37]:
# Validation TFRecord 생성
for idx in val_idx_list:
    bbox_file = bbox_files[idx]
    bbox_path = os.path.join(bbox_dir, bbox_file)
    
    tree = et.parse(bbox_path)
    width = float(tree.find('./size/width').text)
    height = float(tree.find('.size/height').text)
    xmin = float(tree.find('./object/bndbox/xmin').text)
    ymin = float(tree.find('./object/bndbox/ymin').text)
    xmax = float(tree.find('./object/bndbox/xmax').text)
    ymax = float(tree.find('./object/bndbox/ymax').text)
    xc = (xmin + xmax) / 2.
    yc = (ymin + ymax) / 2.
    x = xc / width
    y = yc / height
    w = (xmax - xmin) / width
    h = (ymax - ymin) / height
    
    file_name = os.path.splitext(bbox_file)[0]
    image_file = file_name + '.jpg'
    image_path = os.path.join(image_dir, image_file)
    image = Image.open(image_path)
    image = image.resize((IMG_SIZE, IMG_SIZE))
    bimage = image.tobytes()
    
    class_name = re.sub('_\d+', '', file_name)
    class_num = class2idx[class_name]
    
    if file_name[0].islower():
        bi_cls_num = 0
    else:
        bi_cls_num = 1
    
    
    example = tf.train.Example(features=tf.train.Features(feature={
      'image': _bytes_feature(bimage),
      'cls_num': _int64_feature(class_num),
      'bi_cls_num': _int64_feature(bi_cls_num),
      'x': _float_feature(x),
      'y': _float_feature(y),
      'w': _float_feature(w),
      'h': _float_feature(h)
    }))
    writer_val.write(example.SerializeToString())

writer_val.close()



NameError: name 'writer_val' is not defined